In [19]:
import sys
sys.path.append('..')

import torch
import numpy as np

from utils import ntf
from pipeline import PipelineGED

from transformers import logging
logging.set_verbosity_error()

In [44]:
checkpoints = [
    '../finetuned_models/ner_run_aug_74866/fold0/checkpoint-12195/pytorch_model.bin', 
    # '../finetuned_models/ner_run_aug_74866/fold1/checkpoint-12195/pytorch_model.bin', 
    # '../finetuned_models/ner_run_aug_74866/fold2/checkpoint-12195/pytorch_model.bin', 
    # '../finetuned_models/ner_run_aug_74866/fold3/checkpoint-8130/pytorch_model.bin', 
    # '../finetuned_models/ner_run_aug_74866/fold4/checkpoint-4065/pytorch_model.bin', 
    # '../finetuned_models/ner_run_aug_74866/fold5/checkpoint-8130/pytorch_model.bin', 
    # '../finetuned_models/ner_run_aug_74866/fold6/checkpoint-12195/pytorch_model.bin', 
    # '../finetuned_models/ner_run_aug_74866/fold7/checkpoint-8130/pytorch_model.bin', 
    # '../finetuned_models/ner_run_aug_74866/fold8/checkpoint-8130/pytorch_model.bin', 
    # '../finetuned_models/ner_run_aug_74866/fold9/checkpoint-12195/pytorch_model.bin'
]
model_name = 'hfl/chinese-macbert-base'


device = torch.device("cpu")
# device = torch.device("cuda:0")

data_configs = {
    'model_name':model_name,
    'maxlength':128,
    'train_val_split':-1,
    'test':True, 
    'remove_username':False,
    'remove_punctuation':False, 
    'to_simplified':False, 
    'emoji_to_text':False, 
    'split_words':False, 
    'cut_all':False, 
}
clf = PipelineGED(model_name=model_name, data_configs=data_configs)

In [45]:
## 搭配
texts = [
    '女娲有一双聪明能干的手。', 
    '女娲有一双能干的手。', 
    '女娲有一双聪明的手。', 
]

probs, seq_probs = clf(texts=texts, checkpoints=checkpoints, device=device, output_probabilities=True)
print(probs)

d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()


女娲有一双聪明能干的手。
['手' '。']
女娲有一双能干的手。
[]
女娲有一双聪明的手。
['手' '。']
[[0.34785727 0.6521427 ]
 [0.98132277 0.01867724]
 [0.2169356  0.7830643 ]]


In [47]:
texts = [
    '只有恪守学术道德，才能消除学术腐败，净化学术界。', 
    '只要恪守学术道德，才能消除学术腐败，净化学术界。',
    '只要恪守学术道德，就能消除学术腐败，净化学术界。', 
]

probs, seq_probs = clf(texts=texts, checkpoints=checkpoints, device=device, output_probabilities=True)

d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()


只有恪守学术道德，才能消除学术腐败，净化学术界。
[]
只要恪守学术道德，才能消除学术腐败，净化学术界。
['只' '要' '恪' '守' '学' '术' '道' '德' '，' '才' '能' '消' '除' '学' '术' '腐' '败' '，'
 '净' '化' '学' '术' '界' '。']
只要恪守学术道德，就能消除学术腐败，净化学术界。
['只' '要' '恪' '守' '学' '术' '道' '德' '，' '就' '能' '消' '除' '学' '术' '腐' '败' '，'
 '净' '化' '学' '术' '界' '。']


In [49]:
texts = [
    '我们不仅要在课外学语文，还要在课堂中学语文。', 
    '我们不仅要在课堂中学语文，还要在课外学语文。', 
]

probs, seq_probs = clf(texts=texts, checkpoints=checkpoints, device=device, output_probabilities=True)

d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()


我们不仅要在课外学语文，还要在课堂中学语文。
[]
我们不仅要在课堂中学语文，还要在课外学语文。
[]


In [25]:
# 语义：单面对双面
texts = [
    '他为了民族的兴亡和人民的利益奋斗了一生。', 
    '他为了民族的复兴和人民的利益奋斗了一生。', 
]

probs, seq_probs = clf(texts=texts, checkpoints=checkpoints, device=device, output_probabilities=True)
print(probs)

d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()


他为了民族的兴亡和人民的利益奋斗了一生。
[]
他为了民族的复兴和人民的利益奋斗了一生。
[]
[[0.55955344 0.44044662]
 [0.93049335 0.06950669]]


In [51]:
texts = [
    '我不禁怀疑这条题目是不是老师讲错了。', 
    '我不禁怀疑这道题目是老师讲错了。', 
]

probs, seq_probs = clf(texts=texts, checkpoints=checkpoints, device=device, output_probabilities=True)


d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()


我不禁怀疑这条题目是不是老师讲错了。
[]
我不禁怀疑这道题目是老师讲错了。
[]


In [52]:
texts = [
    '从小明这一个简单的举动中，可以让人看出他是一个细心的好孩子。', 
    '小明这一个简单的举动，可以让人看出他是一个细心的好孩子。', 
]

probs, seq_probs = clf(texts=texts, checkpoints=checkpoints, device=device, output_probabilities=True)

texts = [
    '我想，人是由三部分组成的：对往事的追忆、对未来的憧憬和对现时的把握。', 
    '我想，人是由三部分组成的：对往事的追忆、对现时的把握和对未来的憧憬。', 
]

probs, seq_probs = clf(texts=texts, checkpoints=checkpoints, device=device, output_probabilities=True)

d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()


从小明这一个简单的举动中，可以让人看出他是一个细心的好孩子。
['从' '小' '明' '这' '一' '个' '简' '单' '的' '举' '动' '中' '，' '可' '以' '让' '人' '看'
 '出' '他' '是' '一' '个' '细' '心' '的' '好' '孩' '子' '。']
小明这一个简单的举动，可以让人看出他是一个细心的好孩子。
[]


d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()


我想，人是由三部分组成的：对往事的追忆、对未来的憧憬和对现时的把握。
['由' '：' '的' '、' '对' '的' '和' '的' '。']
我想，人是由三部分组成的：对往事的追忆、对现时的把握和对未来的憧憬。
[]


In [53]:
# 成分残缺/赘余

texts = [
    '时光的流逝不能让我淡去对故乡浓浓的思念，反之，随着年龄的增长，对故乡的思念愈发日久弥坚。', 
    '时光的流逝不能让我淡去对故乡浓浓的思念，反之，随着年龄的增长，对故乡的思念日久弥坚。', 
]

probs, seq_probs = clf(texts=texts, checkpoints=checkpoints, device=device, output_probabilities=True)

d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()


时光的流逝不能让我淡去对故乡浓浓的思念，反之，随着年龄的增长，对故乡的思念愈发日久弥坚。
['时' '光' '的' '流' '逝' '不' '能' '让' '我' '淡' '去' '对' '故' '乡' '浓' '浓' '的' '思'
 '念' '，' '反' '之' '，' '随' '着' '年' '龄' '的' '增' '长' '，' '对' '故' '乡' '的' '思'
 '念' '愈' '发' '日' '久' '弥' '坚' '。']
时光的流逝不能让我淡去对故乡浓浓的思念，反之，随着年龄的增长，对故乡的思念日久弥坚。
['，' '反' '之' '，' '随' '着' '，' '对' '的' '日' '久' '弥' '坚' '。']


In [28]:
texts = [
    '在《京都议定书》第二承诺期问题上，美国继续坚持称，除非中国等新兴市场大国也接受强制减排指标，否则，不接受的话，美国不可能签第二承诺期。', 
    '在《京都议定书》第二承诺期问题上，美国继续坚持称，除非中国等新兴市场大国也接受强制减排指标，否则，美国不可能签第二承诺期。', 
]

probs, seq_probs = clf(texts=texts, checkpoints=checkpoints, device=device, output_probabilities=True)

d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()


在《京都议定书》第二承诺期问题上，美国继续坚持称，除非中国等新兴市场大国也接受强制减排指标，否则，不接受的话，美国不可能签第二承诺期。
['否' '则' '的' '话' '，' '不' '可' '能' '签' '期' '。']
在《京都议定书》第二承诺期问题上，美国继续坚持称，除非中国等新兴市场大国也接受强制减排指标，否则，美国不可能签第二承诺期。
['在' '标' '，' '否' '则' '，' '不' '能' '签' '第' '二' '承' '诺' '期' '。']


In [54]:
texts = [
    '下午三点整，参加典礼的群众全部到齐了。',
     '下午三点整，参加典礼的群众到齐了。',
     '下午三点整，参加典礼的群众全部到了。',
]

probs, seq_probs = clf(texts=texts, checkpoints=checkpoints, device=device, output_probabilities=True)

d:\Develop\chinese-grammar-error-detection\notebooks\..\dataset.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_value = torch.tensor(value[index]).squeeze()


下午三点整，参加典礼的群众全部到齐了。
[]
下午三点整，参加典礼的群众到齐了。
[]
下午三点整，参加典礼的群众全部到了。
[]
